can search for "social media analysis, etc." later and use the code

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
import re
import pickle


# Data Source: truesciphi.org/phi.html, Updated: 2022-May-09 11:03 UTC. Entries: 1095. 
*Be aware of the time!!!*


To be listed, an account must have at least 1,000 Twitter followers and belong to an individual as opposed to a group.

Generally, a Ph.D. in philosophy is required. Exceptions to the Ph.D. rule are made in some cases, e.g., for current Ph.D. students and/or individuals who work in the field and have a substantial following among list members.

Individuals who meet the Ph.D. and follower criteria, but who no longer work in the list field, typically are included in the list. This aids the list in serving as a sort of window on career possibilities.

The list is not necessarily comprehensive. In particular, coverage outside the communities of most existing list members may be poor. Suggestions for additions, including self-nominations, are welcome via Twitter DM. Inclusion of links to Google Scholar pages is helpful.

In [41]:
# df if more suitable for extracting tables

df_full = pd.read_html('philotwitter_full.html') # only one table...
df_women = pd.read_html('philotwitter_women.html') # only one table...
print(len(df_full[0]),len(df_women[0]))

df_full[0].to_csv('extracted_table_{}.csv'.format("full"), header=False)
df_women[0].to_csv('extracted_table_{}.csv'.format("women"), header=False)

1090 366


In [70]:
df_women = pd.read_csv('extracted_table_{}.csv'.format("women"))
df_full = pd.read_csv('extracted_table_{}.csv'.format("full"))

Funtions: Extract information from the html

In [175]:
# username_list,accountname_list = get_name(html)
def get_name(html):
    soup = BeautifulSoup(open(html,encoding='utf-8'))
    username_list = []
    accountname_list = []
    for i in soup.findAll('b'):
        # print(i.text)
        accountname, username = i.text.split("@")
        twt_rul = "https://twitter.com/"+accountname
        username = "@"+username
        # print(username, accountname)
        username_list.append(username)
        accountname_list.append(accountname)

        # want to get the pure text of username, but seems unnecessary
        # only keep normal punctions and characters (consider multilingual!); clean emojis, 
        # print(re.sub("[^A-Za-z0-9_,.\/\ ]",";", username))

    if len(username_list) != len(accountname_list):
        return("error!")
    else:
        return(username_list,accountname_list)



# peer_follower_list,follower_list = get_follower(df)

def get_follower(df):
    follower_list = []
    peer_follower_list = df['Peer followers'].tolist()

    for s in df['Account']:
        result = re.search('user lists(.*)followers', s)
        # print(result.group(1))
        followers = result.group(1)
        # convert to int
        followers = int(followers.replace(",",""))
        follower_list.append(followers)

    # check
    if len(peer_follower_list) != len(follower_list):
        print("error!")
        return([],[])
    else:
        return(peer_follower_list,follower_list)


In [264]:
# time_list = get_time_Twitter(suffix)


def get_time_Twitter(suffix):
    cleaned_list = clean_links(suffix)
    time_list = extract_time(cleaned_list)

    return time_list


# time_list = extract_time(cleaned_list)

def extract_time(cleaned_list):
    time_list = []
    for s in cleaned_list:
        if "years on Twitter" in s:
            idx = s.index("years on Twitter")
            if s[idx-3].isdigit():
                t = s[idx-3:idx-1]
            else:
                t = s[idx-2]
            # Another medthod:
            #     mo = re.match('[^0-9]*([0-9]*) year', s[idx-3:idx+4])
            #     print("year(s):",mo.group(1), "index:",mo.start(1))
        elif "1 year on Twitter" in s:
            t = 1
        else:
            print("error======================================",s)
        
        # check
        if int(t) > 20:
            print(t)
        time_list.append(int(t))

    return(time_list)


# cleaned_list = clean_links(suffix)
def clean_links(suffix):
    df = pd.read_csv('extracted_table_{}.csv'.format(suffix))
    soup = BeautifulSoup(open('philotwitter_{}.html'.format(suffix),encoding='utf-8'))

    link_list = []
    for link in soup.findAll('a'):
        link_list.append(link.text)
    # link_list

    # list = [df['Account'].iloc[0],df['Account'].iloc[1]]
    nolink_list = []
    # print(s)
    for s in df['Account']:
        for link in link_list:
            if link in s:
                s = s[s.index(link)+len(link):]
                continue

        nolink_list.append(s)
        
    cleaned_list = nolink_list

    return(cleaned_list)



Under 1 year => 1 year

In [267]:
# extract_info(suffix)
def extract_info(suffix):
    html = 'philotwitter_{}.html'.format(suffix)
    df = pd.read_csv('extracted_table_{}.csv'.format(suffix))

    username_list,accountname_list = get_name(html)
    peer_follower_list,follower_list = get_follower(df)
    time_list = get_time_Twitter(suffix)
    # make sure to chekc the length
    info_df = pd.DataFrame(dict({
        "username":username_list,
        "accountname":accountname_list,
        "peer follower":peer_follower_list,
        "follower":follower_list,
        "time on Twitter":time_list
    }))
    
    info_df.to_csv('philo_{}_twitter_data.csv'.format(suffix), header=True)
    # info_df
    return info_df



In [268]:
extract_info("full")
extract_info("women")

,username,accountname,peer follower,follower,time on Twitter
0,@ranilillanjum,Dr Rani Lill Anjum ☮️,718,15178,12
1,@Helenreflects,Helen De Cruz,655,18440,8
2,@kate_manne,Kate Manne,651,55441,7
3,@anitaleirfall,Anita Leirfall,598,20052,11
4,@Skye_Cleary,Dr Skye Cleary,584,14207,9
...,...,...,...,...,...
360,@AlbaLafarga,Alba Lafarga,9,2215,11
361,@silvialgil,Silvia L. Gil,7,1226,10
362,@EveKeneinan,Eve Keneinan 𝛗☦️ن,6,8876,11
363,@xiscahomar,Xisca Homar,6,1034,11


In [269]:
df_full = pd.read_csv('philo_full_twitter_data.csv')
df_women = pd.read_csv('philo_women_twitter_data.csv')



In [271]:
#  make sure that all women are in the full list => we use interscetion
women_username_list = set(df_women['username'].tolist()) & set(df_full['username'].tolist())
n = len(df_full)
df_all = df_full
df_all['gender'] = "men/nonbinary"

for i in range(n):
    if df_full['username'].iloc[i] in women_username_list:
        df_all['gender'].iloc[i] = "women"

df_all

c:\Users\zhenj\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,Unnamed: 0,username,accountname,peer follower,follower,time on Twitter,gender
0,0,@GreggDCaruso,Gregg Caruso,769,23619,8,men/nonbinary
1,1,@ranilillanjum,Dr Rani Lill Anjum ☮️,718,15178,12,women
2,2,@DailyNousEditor,Daily Nous / Justin Weinberg,706,21771,8,men/nonbinary
3,3,@CliffordSosis,What Is It Like to Be a Philosopher?,697,16168,10,men/nonbinary
4,4,@csandis,Constantine Sandis,688,11808,11,men/nonbinary
...,...,...,...,...,...,...,...
1084,1084,@EveKeneinan,Eve Keneinan 𝛗☦️ن,6,8876,11,women
1085,1085,@xiscahomar,Xisca Homar,6,1034,11,women
1086,1086,@filosofiaminuto,Juan Denis // Filosofía en Minutos,5,9278,6,men/nonbinary
1087,1087,@JasonWerbeloff,Jason Werbeloff,5,1142,12,men/nonbinary


In [272]:
df_all.to_csv('{}_philo_twitter_data.csv'.format("all"), header=True)

Other information (TBD)

In [183]:
# get the links: including twitter page (@accountname), personal website and/or google scholar page 

soup_women = BeautifulSoup(open("philotwitter_women.html",encoding='utf-8'))


extract_philotwitter_info("philotwitter_full.html")


def extract_links(html):
    soup = BeautifulSoup(open(html,encoding='utf-8'))

    links = []
    for link in soup.findAll('a'):
        links.append(link.text)


NameError: name 'extract_philotwitter_info' is not defined

In [ ]:

pickle.dump( r1, open( "philosophen_liste.p", "wb" ) )

In [ ]:

philos = pickle.load( open( "philosophen_liste.p", "rb" ) )
print(philos)